### Combine Data Sets 

This notebook combines the following:  
*    ACS data sets DP02, DP03, DP04, and DP05   
*    Municipal Tax data  
*    Sustainability New Jersey data
*    Train stations
*    Indeed.com job listings

#### Only run the sections that build on the already existing pickles


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sb

In [ ]:
## Import pickled data

econ = pd.read_pickle('econ.pkl')
housing =pd.read_pickle('housing.pkl')
demog = pd.read_pickle('AgeSexRace.pkl')
misc = pd.read_pickle('misc.pkl')

In [ ]:
# join data frames into a combined dataframe

econ_housing = pd.merge(econ, housing, on=['FIPS','city_town'])
demog_misc = pd.merge(demog, misc, on=['FIPS','city_town'])
acs = pd.merge(econ_housing, demog_misc, on=['FIPS','city_town'])

In [ ]:
# delete rows where 0 percent in labor force is indicated
acs = acs[acs['p_in_lab'] != 0]
acs.head()
acs.info()

In [ ]:
 acs['ix']=range(565)

In [ ]:

acs = acs.set_index('ix')
acs.iloc[0:4]
acs.info()
acs.index[:10]

In [ ]:
## break out town and county names
s =pd.DataFrame([x.split(sep=',') for x in acs['city_town']])
s.columns = ['town','county','state']
s.info()

In [ ]:
acs_s = s.join(acs)
acs_s.iloc[0:3]

In [ ]:
acs_s.to_pickle('acs_s.pkl')

### Below, we add the taxes data

In [42]:
import pandas as pd
taxes = pd.read_pickle('taxes.pkl')
acs_s = pd.read_pickle('acs_s.pkl')

In [43]:
acs_s.iloc[:,:6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 6 columns):
town         565 non-null object
county       565 non-null object
state        565 non-null object
FIPS         565 non-null object
city_town    565 non-null object
p_in_lab     565 non-null float64
dtypes: float64(1), object(5)
memory usage: 26.6+ KB


In [44]:
taxes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Data columns (total 20 columns):
FIPS                                                           565 non-null object
CountyLevy%ofTotalLevy                                         565 non-null float64
SchoolLevy%ofTotalLevy                                         565 non-null float64
MunicipalLevy%ofTotalLevy                                      565 non-null float64
CYMunicipalRate                                                565 non-null float64
CYMunicipalLibraryRate                                         565 non-null float64
CYMunicipalOpenSpaceRate                                       565 non-null float64
CYTotalMunicipalRate                                           565 non-null float64
CYSchoolRate                                                   565 non-null float64
CYCountyRate                                                   565 non-null float64
CYTotalRate                                             

In [45]:
acs_tax = acs_s.merge(taxes, on='FIPS')

In [46]:
acs_tax.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Columns: 154 entries, town to CYTotalEQRate(REAPNotIncluded)
dtypes: float64(147), int64(2), object(5)
memory usage: 684.2+ KB


In [47]:
acs_tax.to_pickle('acs_tax.pkl')

In [48]:
!ls *.pkl

acs_s.pkl	 d2.pkl       labor_combine.pkl  muni.pkl
acs_tax.pkl	 econ.pkl     labor.pkl		 njtrain_stations.pkl
acs_tax_sus.pkl  housing.pkl  misc.pkl		 sustain.pkl
AgeSexRace.pkl	 jobs.pkl     muni_names.pkl	 taxes.pkl


In [50]:
sustain = pd.read_pickle('sustain.pkl')

In [51]:
## This merge adds only one column, num_badge, which represents efforts towards sustainability
## as recognized by Sustainability New Jersey

acs_tax_sus = acs_tax.merge(sustain, on='FIPS')

In [52]:
acs_tax_sus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Columns: 155 entries, town to num_badge
dtypes: float64(148), int64(2), object(5)
memory usage: 688.6+ KB


In [53]:
acs_tax_sus.to_pickle('acs_tax_sus.pkl')

In [54]:
acs_tax_sus.columns

Index(['town', 'county', 'state', 'FIPS', 'city_town', 'p_in_lab',
       'p_unemployed', 'p_fem_labforce', 'p_fem_employed', 'p_ch6_paremp',
       ...
       'AverageResidentialPropertyValue',
       'AverageTotalPropertyTaxes(notincludingcreditsanddeductions)',
       'StateEqualizationTableAverageRatio',
       'StateEqualizationTableAverageRatio(DecimalForm)', 'CYCountyEQTaxRate',
       'CYSchoolEQTaxRate', 'CYLocalPurposeMunicipalEQTaxRate',
       'CYTotalMunicipalEQRate', 'CYTotalEQRate(REAPNotIncluded)',
       'num_badge'],
      dtype='object', length=155)

In [55]:
acs_tax_sus.loc[:,'FIPS':'city_town'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Data columns (total 2 columns):
FIPS         565 non-null object
city_town    565 non-null object
dtypes: object(2)
memory usage: 13.2+ KB


In [56]:
njtrain = pd.read_pickle('njtrain_stations.pkl')
#njtrain = njtrain.loc[:,['FIPS','num_train_stat']]
njtrain.info()
njtrain.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 2 columns):
FIPS              565 non-null object
num_train_stat    565 non-null float64
dtypes: float64(1), object(1)
memory usage: 8.9+ KB


,FIPS,num_train_stat
0,3402500070,0.0
1,3400100100,1.0
2,3401900550,0.0
3,3404100670,0.0
4,3400300700,1.0


In [57]:
d1 = acs_tax_sus.merge(njtrain, on='FIPS')
d1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Columns: 156 entries, town to num_train_stat
dtypes: float64(149), int64(2), object(5)
memory usage: 693.0+ KB


In [58]:
d1.head()

,town,county,state,FIPS,city_town,p_in_lab,p_unemployed,p_fem_labforce,p_fem_employed,p_ch6_paremp,...,AverageTotalPropertyTaxes(notincludingcreditsanddeductions),StateEqualizationTableAverageRatio,StateEqualizationTableAverageRatio(DecimalForm),CYCountyEQTaxRate,CYSchoolEQTaxRate,CYLocalPurposeMunicipalEQTaxRate,CYTotalMunicipalEQRate,CYTotalEQRate(REAPNotIncluded),num_badge,num_train_stat
0,Absecon city,Atlantic County,New Jersey,3400100100,"Absecon city, Atlantic County, New Jersey",67.3,7.0,61.1,54.9,78.2,...,4539,92.04,0.9204,0.463,1.444,0.945,0.978,2.885,0.0,1.0
1,Atlantic City city,Atlantic County,New Jersey,3400102080,"Atlantic City city, Atlantic County, New Jersey",60.4,10.5,57.6,48.1,61.6,...,5315,87.47,0.8747,0.336,1.084,1.528,1.573,2.993,2.0,1.0
2,Brigantine city,Atlantic County,New Jersey,3400107810,"Brigantine city, Atlantic County, New Jersey",61.4,5.5,57.0,52.8,92.3,...,4455,94.38,0.9438,0.509,0.492,0.671,0.671,1.672,2.0,0.0
3,Buena borough,Atlantic County,New Jersey,3400108680,"Buena borough, Atlantic County, New Jersey",68.4,12.5,61.1,55.2,74.7,...,4090,113.03,1.1303,0.502,1.550,1.055,1.055,3.107,1.0,0.0
4,Buena Vista township,Atlantic County,New Jersey,3400108710,"Buena Vista township, Atlantic County, New Jersey",63.0,9.8,61.6,52.8,60.0,...,4017,106.30,1.0630,0.489,1.558,0.419,0.419,2.466,3.0,0.0


In [59]:
labor = pd.read_pickle('labor_combine.pkl')
labor.info()
labor.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Data columns (total 4 columns):
FIPS               565 non-null object
management_rate    565 non-null float64
teacher_rate       565 non-null float64
developer_rate     565 non-null float64
dtypes: float64(3), object(1)
memory usage: 22.1+ KB


,FIPS,management_rate,teacher_rate,developer_rate
0,3400100100,2.279898,0.727648,0.997808
1,3400102080,2.698644,0.359848,0.359848
2,3400107810,1.021209,0.000000,0.000000
3,3400108680,1.523245,0.335447,0.000000
4,3400108710,0.000000,0.000000,0.000000


In [60]:
d2 = d1.merge(labor, on='FIPS')

In [61]:
d2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 0 to 564
Columns: 159 entries, town to developer_rate
dtypes: float64(152), int64(2), object(5)
memory usage: 706.2+ KB


In [62]:
d2.to_pickle('d2.pkl')